In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import csv
import os
from os.path import isfile, join
import re
import h5py
import random
import numpy as np
import h5py

import tensorflow as tf
import keras
from keras.models import Sequential
from keras.models import Model
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.optimizers import Adam
from keras.layers import Input, Flatten, Dense, Dropout,  Activation, Conv3D, MaxPooling3D, UpSampling3D, BatchNormalization, Activation, Reshape
from keras.layers import Conv2D, MaxPooling2D

from keras.optimizers import SGD
from keras.layers.merge import concatenate, add

from keras.utils import plot_model
from keras import optimizers


from os import listdir
from random import shuffle
from scipy.ndimage import gaussian_filter

In [5]:
def video_predict(video_input, video_output, sigma, limit):
    
    # cargamos la estructura de la red y sus pesos  
    dirmodel = '/mnt/MD1200A/lconcha/videos/Modelo_reconocimiento_automático/modelo/'
    
    with open(dirmodel + "data_augmentation.json") as json_file:
        json_config = json_file.read()
    model = keras.models.model_from_json(json_config)

    checkpoint_path = "/mnt/MD1200A/lconcha/videos/Modelo_reconocimiento_automático/modelo/cp-0012.ckpt"
    checkpoint_dir = os.path.dirname(checkpoint_path)
    model.load_weights(checkpoint_path)
    
    
    #Cargamos el video que queremos meter como input a la red y realizamos una prediccion
    cap = cv2.VideoCapture(video_input)
    frames = [] #almacenamos los frames del video


    while (cap.isOpened()):
        ret, frame = cap.read()
        if ret == True:
            resize = cv2.resize(frame, (128,128), interpolation = cv2.INTER_AREA)
            frames.append(resize)

        if ret == False:
            break

    it = 0
    etiq = [] #almacenaremos tan

    for line in frames: 
        etiq.append(it)
        it+=1


    groups8 = []
    boolean= False
    pos = 0

    while boolean == False:

        for line in etiq:
            if line== (0 + pos) or line == (4 + pos) or line == (8 + pos) or line == (12 + pos) or line == (16 + pos) or line == (20 + pos) or line == (24 + pos) or line == (28 + pos):
                groups8.append(frames[line])


        pos += 1
        if pos == len(frames) - 32:
            boolean = True

    sets = [] #En esta lista vamos a separar en conjuntos de 8 los frames obtenidos anteriormente
    X_values = [] #En esta lista cada conjunto de 8 frames lo convertiremos en un array de shape [8 128 128]

    for i in range(0, len(groups8), 8):
        sets.append(groups8[i:i+8])

    for j in sets: #recorremos sets y concatenamos los 8 frames, para añadirlos a una nueva lista
        X = np.stack(j)
        X_values.append(X)


    x_array = np.array(X_values)

    prediction = model.predict(x_array)

    array_prediction = np.argmax(prediction, axis=1)
    
    
    # Una vez hecha la prediccion, vemos las posiciones en las que la red ha determinado que la rata estaba en estado ataque
    pos_at = []

    posicion = 0
    for i in array_prediction:
        if i == 1:
            pos_at.append(posicion)
        posicion +=1

    b = []
    for i in range(32):
        b.append((pos_at[-1])+(i+1))

    c = pos_at + b
    c = np.array(c)
    
    #Creamos un nuevo video en el que se ven los frames etiquetados con el estado correspondiente de la rata
    
    fps = 10.0 #Frames por segundo en la reproduccion
    
    video_label = cv2.VideoWriter(video_output + '.mp4' , cv2.VideoWriter_fourcc(*"XVID"),fps,(720,720))

    cap = cv2.VideoCapture(video_input)

    it = 0
    while (cap.isOpened()):
        ret, frame = cap.read()
        if ret == True:

            if it == len(prediction):
                it -=1

            if it in c:
                resize = cv2.resize(frame, (720,720), interpolation = cv2.INTER_AREA)
                text = cv2.putText(resize, "Estado: Ataque" + ' ' + str(round((prediction[it])[1]*100,2)) +'%', (10,70), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 4, cv2.LINE_AA)
                video_label.write(text)
            else:
                resize = cv2.resize(frame, (720,720), interpolation = cv2.INTER_AREA)
                text = cv2.putText(resize, "Estado: Reposo" + ' ' + str(round((prediction[it])[0]*100,2))+'%', (10,70), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 4, cv2.LINE_AA)
                video_label.write(text)


            it +=1
        if ret == False:
            break

    video_label.release()
    
    
    
    ################################################# GAUSSIAN FILTER   ##############################################
    
    array_prediction = array_prediction.astype(np.float)
    
    filter_gaus = gaussian_filter(array_prediction, sigma)
    
    pos_at_gaus = []

    posicion = 0
    for i in filter_gaus:
        if i > limit:
            pos_at_gaus.append(posicion)
        posicion +=1

    b_gaus = []
    for i in range(32):
        b_gaus.append((pos_at_gaus[-1])+(i+1))

    c_gaus = pos_at_gaus + b_gaus
    c_gaus = np.array(c_gaus)

      
    video_label = cv2.VideoWriter(video_output + 'gaussianFilter.mp4', cv2.VideoWriter_fourcc(*"XVID"),fps,(720,720))

    cap = cv2.VideoCapture(video_input)
    
    frame_atack = []
    it = 0
    while (cap.isOpened()):
        ret, frame = cap.read()
        if ret == True:
            if it in c_gaus:
                resize = cv2.resize(frame, (720,720), interpolation = cv2.INTER_AREA)
                text = cv2.putText(resize, "Estado: Ataque", (10,70), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 0, 255), 4, cv2.LINE_AA)
                video_label.write(text)
                frame_atack.append(it)
                
            else:
                resize = cv2.resize(frame, (720,720), interpolation = cv2.INTER_AREA)
                text = cv2.putText(resize, "Estado: Reposo", (10,70), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 0, 255), 4, cv2.LINE_AA)
                video_label.write(text)

            it +=1
        if ret == False:
            break

    video_label.release()
    
##################################### archivo csv ######################


#COMPROBAMOS NUMERO DE ATAQUES

    consec = np.diff(frame_atack) #comprueba si son correlativos los numeros
    
    posicion = []
    pos = 0 
    for i in consec:
        if i !=1: #si algun numero es distinto a 1 quiere decir que se pierde la correlacion con el siguiente
            posicion.append(pos)
        pos +=1
        
    #en el caso de que el video solo disponga de un ataque:
    if len(posicion) == 0:
        print('Detectado un ataque')
        
        start = frame_atack[0]/fps
        end = frame_atack[-1]/fps

        minutes_start = int(start/60)
        seconds_start = start%60

        minutes_end = int(end/60)
        seconds_end = end%60

        it = 0
        pos = []
        for i in video_input:
            if i == '/':
                pos.append(it)
            it += 1

        name = video_input[(pos[-1])+1:]

        text = [name, str(minutes_start) + ':' + str(round(seconds_start)), str(minutes_end) + ':' + str(round(seconds_end))]

        header = ['Video Name','START', 'END']

        with open(video_output + '.csv', 'wt') as f:
            writer = csv.writer(f)
            writer.writerow(header)
            writer.writerow(text)
            
        print('Documento creado')
    
    
    
    #En el caso de que tenga mas de un ataque:
    nuevo_array = []
    if len(posicion) >= 1:
        print('Detectados varios ataques')
        for i in posicion:
            nuevo_array.append(frame_atack[0])
            nuevo_array.append(frame_atack[i])
            nuevo_array.append(frame_atack[i+1])
            nuevo_array.append(frame_atack[-1])
            
        nuevo_array = set(nuevo_array) #quitamos duplicados
        nuevo_array=sorted(list(nuevo_array)) #ordena y lo transforma en lista
        
        it = 0
        pos = []
        for i in video_input:
            if i == '/':
                pos.append(it)
            it += 1

        name = video_input[(pos[-1])+1:]
        
        header = ['Video Name','START', 'END']

        with open(video_output + '.csv', 'wt') as f:
            writer = csv.writer(f)
            writer.writerow(header)

            for i in range (0, len(nuevo_array), 2):
                start = nuevo_array[i]/fps
                end = nuevo_array[i+1]/fps

                minutes_start = int(start/60)
                seconds_start = start%60

                minutes_end = int(end/60)
                seconds_end = end%60

                text = [name, str(minutes_start) + ':' + str(round(seconds_start)), str(minutes_end) + ':' + str(round(seconds_end))]
                writer.writerow(text)
                
        print('Documento creado')
            
    print('Videos creados')

In [6]:
video_predict('/mnt/MD1200A/lconcha/videos/RAT_VIDEOS_CROPPED_RESIZE/1.2018-1-9.Rat9.mp4', '/mnt/MD1200A/lconcha/videos/1.Rat9', 50,0.25)

Detectado un ataque
Documento creado
[2763, 2764, 2765, 2766, 2767, 2768, 2769, 2770, 2771, 2772, 2773, 2774, 2775, 2776, 2777, 2778, 2779, 2780, 2781, 2782, 2783, 2784, 2785, 2786, 2787, 2788, 2789, 2790, 2791, 2792, 2793, 2794, 2795, 2796, 2797, 2798, 2799, 2800, 2801, 2802, 2803, 2804, 2805, 2806, 2807, 2808, 2809, 2810, 2811, 2812, 2813, 2814, 2815, 2816, 2817, 2818, 2819, 2820, 2821, 2822, 2823, 2824, 2825, 2826, 2827, 2828, 2829, 2830, 2831, 2832, 2833, 2834, 2835, 2836, 2837, 2838, 2839, 2840, 2841, 2842, 2843, 2844, 2845, 2846, 2847, 2848, 2849, 2850, 2851, 2852, 2853, 2854, 2855, 2856, 2857, 2858, 2859, 2860, 2861, 2862, 2863, 2864, 2865, 2866, 2867, 2868, 2869, 2870, 2871, 2872, 2873, 2874, 2875, 2876, 2877, 2878, 2879, 2880, 2881, 2882, 2883, 2884, 2885, 2886, 2887, 2888, 2889, 2890, 2891, 2892, 2893, 2894, 2895, 2896, 2897, 2898, 2899, 2900, 2901, 2902, 2903, 2904, 2905, 2906, 2907, 2908, 2909, 2910, 2911, 2912, 2913, 2914, 2915, 2916, 2917, 2918, 2919, 2920, 2921, 2922, 29